# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.

In [112]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import seaborn
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
%matplotlib inline

In [113]:
# Make a request
url_main = 'https://www.topuniversities.com'  # Found with postman
r = requests.get(url_main + '/sites/default/files/qs-rankings-data/357051.txt')
print('Response status code: {0}\n'.format(r.status_code))
page_body = r.text

# Serialize the json data with json library
rank_json = json.loads(page_body)

Response status code: 200



In [114]:
rank_df = pd.DataFrame()
rank_df = rank_df.from_dict(rank_json['data']).head(200)
rank_df.stars
rank_df.drop(['logo', 'stars', 'nid','cc', 'score'], axis=1, inplace=True)
rank_df.set_index('core_id', inplace=True)
rank_df = rank_df[['title', 'rank_display', 'country', 'region', 'url']]
rank_df.head()

,title,rank_display,country,region,url
core_id,,,,,
410,Massachusetts Institute of Technology (MIT),1,United States,North America,/universities/massachusetts-institute-technolo...
573,Stanford University,2,United States,North America,/universities/stanford-university
253,Harvard University,3,United States,North America,/universities/harvard-university
94,California Institute of Technology (Caltech),4,United States,North America,/universities/california-institute-technology-...
95,University of Cambridge,5,United Kingdom,Europe,/universities/university-cambridge


In [115]:
def my_find(html_attributes, new_df_column_name, rank_df):
    _tag = html_attributes['tag']
    _class = html_attributes['class']
    # _list is a temporary list that will store the values found and then be converted in the df to be returned.
    _list = []
    for url in rank_df.url:
        # for every url contained in rank_df['url'], perform the corresponding request:
        uni_url = requests.get(url_main + url)
        uni_body = uni_url.text
        soup = BeautifulSoup(uni_body, 'html.parser')
        # look for <tag=_tag, class=_class>
        soup1 = soup.find(_tag, class_=_class)
        # if such tag has been found, look then for <tag=_tag, class='number> where the value 
        # we're interested in is stored! otherwise append -99
        if soup1:
            soup2 = soup1.find(_tag, class_='number') 
            # if such tag has been found, append its value to the _list, otherwise append -99
            if soup2:
                _list.append({new_df_column_name: soup2.text})
            else:
                _list.append({new_df_column_name: -99})
        else:
            _list.append({new_df_column_name: -99})
    # convert _list to dataframe and return it
    return pd.DataFrame.from_dict(_list).replace({r'\n': ''},\
                                                 regex=True).replace({r',': ''},\
                                                                     regex=True).apply(pd.to_numeric).astype(int)

In [116]:
# defining HTML tag and class attributes that we want to find
tofind = [{'tag':'div', 'class': 'total faculty'}, 
          {'tag':'div', 'class': 'inter faculty'}, 
          {'tag':'div', 'class': 'total student'}, 
          {'tag':'div', 'class': 'total inter'}]

# creating DataFrame with the data found (NaN values = -99)
details_df = pd.concat([my_find(tofind[0], 'fac_memb_tot', rank_df),
                        my_find(tofind[1], 'fac_memb_int', rank_df),
                        my_find(tofind[2], 'nb_stud_tot', rank_df),
                        my_find(tofind[3], 'nb_stud_int', rank_df)], axis=1)

# concatenate the DataFrames into a unique one
details_df.set_index(rank_df.index, inplace=True)
QS_df = pd.concat([rank_df, details_df], axis=1)

# cleaning the unique DataFrame (deleting the = in rank_display)
QS_df.drop(['url'], axis=1, inplace=True)
QS_df.rank_display = QS_df.rank_display.replace({r'=': ''}, regex=True).apply(pd.to_numeric).astype(int)

# Creating the faculty_members_ratio and number_of_students ratio
QS_df['fac_memb_ratio'] = QS_df.fac_memb_tot / QS_df.nb_stud_tot
QS_df['int_stud_ratio'] = QS_df.nb_stud_int / QS_df.nb_stud_tot

# Deleting what's useless
del details_df, tofind, rank_df, r, page_body, rank_json

In [117]:
# resetting the index, dropping the 'core_id' column wich is useless, and renaming the columns
# in order to prepare themerge with the corresponding TimesHigherEducation dataframe
QS_df.reset_index(inplace = True)
QS_df.drop('core_id', axis=1, inplace = True)
QS_df.columns = ['QS_name', 'QS_rank', 'country', 'region', 'QS_fac_memb_tot', 'QS_fac_memb_int',
                'QS_nb_stud_tot', 'QS_nb_stud_int', 'QS_fac_memb_ratio', 'QS_int_stud_ratio']
QS_df.head()

,QS_name,QS_rank,country,region,QS_fac_memb_tot,QS_fac_memb_int,QS_nb_stud_tot,QS_nb_stud_int,QS_fac_memb_ratio,QS_int_stud_ratio
0,Massachusetts Institute of Technology (MIT),1,United States,North America,2982,1679,11067,3717,0.269450,0.335863
1,Stanford University,2,United States,North America,4285,2042,15878,3611,0.269870,0.227422
2,Harvard University,3,United States,North America,4350,1311,22429,5266,0.193945,0.234785
3,California Institute of Technology (Caltech),4,United States,North America,953,350,2255,647,0.422616,0.286918
4,University of Cambridge,5,United Kingdom,Europe,5490,2278,18770,6699,0.292488,0.356899


In [118]:
QS_df.head(10)

,QS_name,QS_rank,country,region,QS_fac_memb_tot,QS_fac_memb_int,QS_nb_stud_tot,QS_nb_stud_int,QS_fac_memb_ratio,QS_int_stud_ratio
0,Massachusetts Institute of Technology (MIT),1,United States,North America,2982,1679,11067,3717,0.269450,0.335863
1,Stanford University,2,United States,North America,4285,2042,15878,3611,0.269870,0.227422
2,Harvard University,3,United States,North America,4350,1311,22429,5266,0.193945,0.234785
3,California Institute of Technology (Caltech),4,United States,North America,953,350,2255,647,0.422616,0.286918
4,University of Cambridge,5,United Kingdom,Europe,5490,2278,18770,6699,0.292488,0.356899
5,University of Oxford,6,United Kingdom,Europe,6750,2964,19720,7353,0.342292,0.372870
6,UCL (University College London),7,United Kingdom,Europe,6345,2554,31080,14854,0.204151,0.477928
7,Imperial College London,8,United Kingdom,Europe,3930,2071,16090,8746,0.244251,0.543567
8,University of Chicago,9,United States,North America,2449,635,13557,3379,0.180645,0.249244
9,ETH Zurich - Swiss Federal Institute of Techno...,10,Switzerland,Europe,2477,1886,19815,7563,0.125006,0.381681


# TIMES HIGHER EDUCATION

In [119]:
# Making the request, beautifully soupping for extracting the dataframe
URL = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/'\
                +'world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r = requests.get(URL)
print('r = {r} // status_code = {status}'.format(r=r,status=r.status_code))
r.content
soupp = BeautifulSoup(r.content,'html.parser')
rank_json = json.loads(r.text)
THE_df = pd.DataFrame()
THE_df = THE_df.from_dict(rank_json['data']).head(200)
THE_df.head()

r = <Response [200]> // status_code = 200


,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,...,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,...,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,...,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,...,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,...,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,...,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...


In [120]:
# select columns of our interest and changing their name
THE_df = THE_df[['name', 'aliases', 'location', 'rank', 'stats_number_students', 'stats_student_staff_ratio', 'stats_pc_intl_students']]
THE_df.columns=['THE_name', 'THE_aliases', 'THE_location', 'THE_rank','THE_nb_stud_tot', 'THE_stats_student_staff_ratio','THE_int_stud_ratio']
THE_df.head()

,THE_name,THE_aliases,THE_location,THE_rank,THE_nb_stud_tot,THE_stats_student_staff_ratio,THE_int_stud_ratio
0,University of Oxford,University of Oxford,United Kingdom,1,"20,409",11.2,38%
1,University of Cambridge,University of Cambridge,United Kingdom,2,"18,389",10.9,35%
2,California Institute of Technology,California Institute of Technology caltech,United States,=3,"2,209",6.5,27%
3,Stanford University,Stanford University,United States,=3,"15,845",7.5,22%
4,Massachusetts Institute of Technology,Massachusetts Institute of Technology,United States,5,"11,177",8.7,34%


# still the usual, hard-to-understand cleaning of the dataset

In [121]:
# THE_int_stud_ratio
THE_df['THE_int_stud_ratio'] = THE_df['THE_int_stud_ratio'].str.replace('%', '').astype('double')/100

# THE_nb_stud_tot
THE_df['THE_nb_stud_tot'] = THE_df['THE_nb_stud_tot'].str.replace(',', '').astype('int')

# THE_fac_memb_int is a missing data, I add it just to state it clearly
THE_df['THE_fac_memb_int'] = -99

# THE_fac_memb_tot
THE_df['THE_stats_student_staff_ratio']=THE_df['THE_stats_student_staff_ratio'].astype(float)
THE_df['THE_fac_memb_tot'] = THE_df['THE_nb_stud_tot']/THE_df['THE_stats_student_staff_ratio']
THE_df['THE_fac_memb_tot'] = THE_df['THE_fac_memb_tot'].astype(int)

# THE_nb_stud_int
THE_df['THE_nb_stud_int'] = THE_df['THE_nb_stud_tot']*THE_df['THE_int_stud_ratio']
THE_df['THE_nb_stud_int'] = THE_df['THE_nb_stud_int'].astype(int)

# THE_fac_memb_ratio
THE_df['THE_fac_memb_ratio'] = 1/THE_df['THE_stats_student_staff_ratio']
THE_df=THE_df.drop('THE_stats_student_staff_ratio', axis=1)

# THE_rank
THE_df['THE_rank'] = THE_df['THE_rank'].astype(str)
THE_df['THE_rank'] = THE_df['THE_rank'].replace({r'=': ''}, regex=True).apply(pd.to_numeric).astype(int)

# let's now have a look at the clean THE_df
THE_df.head()

,THE_name,THE_aliases,THE_location,THE_rank,THE_nb_stud_tot,THE_int_stud_ratio,THE_fac_memb_int,THE_fac_memb_tot,THE_nb_stud_int,THE_fac_memb_ratio
0,University of Oxford,University of Oxford,United Kingdom,1,20409,0.38,-99,1822,7755,0.089286
1,University of Cambridge,University of Cambridge,United Kingdom,2,18389,0.35,-99,1687,6436,0.091743
2,California Institute of Technology,California Institute of Technology caltech,United States,3,2209,0.27,-99,339,596,0.153846
3,Stanford University,Stanford University,United States,3,15845,0.22,-99,2112,3485,0.133333
4,Massachusetts Institute of Technology,Massachusetts Institute of Technology,United States,5,11177,0.34,-99,1284,3800,0.114943


#### Note that the aliases is always more precise than the name

In [122]:
THE_df.head(5)

,THE_name,THE_aliases,THE_location,THE_rank,THE_nb_stud_tot,THE_int_stud_ratio,THE_fac_memb_int,THE_fac_memb_tot,THE_nb_stud_int,THE_fac_memb_ratio
0,University of Oxford,University of Oxford,United Kingdom,1,20409,0.38,-99,1822,7755,0.089286
1,University of Cambridge,University of Cambridge,United Kingdom,2,18389,0.35,-99,1687,6436,0.091743
2,California Institute of Technology,California Institute of Technology caltech,United States,3,2209,0.27,-99,339,596,0.153846
3,Stanford University,Stanford University,United States,3,15845,0.22,-99,2112,3485,0.133333
4,Massachusetts Institute of Technology,Massachusetts Institute of Technology,United States,5,11177,0.34,-99,1284,3800,0.114943


## Those where our dataset, from now we try to merge

In [123]:
QS_df['country'].value_counts().index

Index(['United States', 'United Kingdom', 'Germany', 'Netherlands', 'Japan',
       'Australia', 'Canada', 'Switzerland', 'China', 'South Korea', 'France',
       'Hong Kong', 'Sweden', 'Belgium', 'Italy', 'Spain', 'India', 'Denmark',
       'Singapore', 'Brazil', 'Taiwan', 'Finland', 'Norway', 'Austria',
       'New Zealand', 'Chile', 'Mexico', 'Ireland', 'Russia', 'South Africa',
       'Argentina', 'Saudi Arabia', 'Malaysia', 'Israel'],
      dtype='object')

In [124]:
QS_US = QS_df['country'][QS_df['country']=='United States'].value_counts()['United States']
THE_US = THE_df['THE_location'][THE_df['THE_location']=='United States'].value_counts()['United States']
abs(THE_US-QS_US)

15

In [132]:
# Set russian federation equal russia for comparing countries
THE_df['THE_location'][THE_df['THE_location']=='Russian Federation']='Russia'

# find differences in listed countries
tmp1 = QS_df['country'].value_counts().index
tmp2 = THE_df['THE_location'].value_counts().index
my_diff = list(set(tmp1).symmetric_difference(set(tmp2)))
my_diff

/home/zampieri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


['Brazil',
 'Saudi Arabia',
 'Luxembourg',
 'Malaysia',
 'Mexico',
 'Chile',
 'Israel',
 'Argentina',
 'India']

In [171]:
list(THE_df)
out3 = pd.DataFrame()
#out3.columns.values = list(THE_df)
out_3 = THE_df.iloc[0:0,:] 
out_3

,THE_name,THE_aliases,THE_location,THE_rank,THE_nb_stud_tot,THE_int_stud_ratio,THE_fac_memb_int,THE_fac_memb_tot,THE_nb_stud_int,THE_fac_memb_ratio


In [190]:
# return dataframes df1,df2 with only the rows where index in column1 exist also in column2 and vice versa
def exclude_differences(df1,df2,column1,column2):
    tmp1 = df1[column1].value_counts().index
    tmp2 = df2[column2].value_counts().index
    my_diff = list(set(tmp1).symmetric_difference(set(tmp2)))
    out_1 = df1.copy()
    out_2 = df2.copy()
    out_3 = out_1.iloc[0:0,:].copy()
    unmatched_df = pd.DataFrame()
    
    for loc in my_diff:
        out_31 = out_1[out_1[column1] == loc] # we wanna keep track of non meargeable elements
        out_32 = out_2[out_2[column2] == loc]
        out_3 = pd.concat([out_3, out_31])
        out_3 = pd.concat([out_3, out_32])
        #out_3.append(out_31)
        
        out_1 = out_1[out_1[column1] != loc] # now we get rid of these rows
        out_2 = out_2[out_2[column2] != loc]
        
        
        
    return out_1, out_2, out_3

In [193]:
QS_tmp, THE_tmp, non_mergeable = exclude_differences(QS_df,THE_df,'country','THE_location')

print('QS: ',QS_tmp.shape[0],'    THE: ',THE_tmp.shape[0])
print(non_mergeable.shape[0])

QS_tmp.sort_values('country',inplace=True)
THE_tmp.sort_values('THE_location',inplace=True)

QS:  187     THE:  199
14


##### Note that non_meargeable contains only a subset of all the non-meargeables rows

In [194]:
# just a cross check
tmp1 = QS_tmp['country'].value_counts().sort_index().index
tmp2 = THE_tmp['THE_location'].value_counts().sort_index().index
print(tmp1 == tmp2)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


In [ ]:
if 

In [195]:
QS = QS_tmp.copy()
THE = THE_tmp.copy()

#initializing a new column of the THE_df with the probable, corresponding QS name
THE['THE_corresponding QS name']='unknown'

for loc in THE['THE_location']:
    QS_name = list(QS.QS_name[QS['country']==loc])
    THE_name = list(THE.THE_name[THE['THE_location']==loc])
    
    #finding the probable corresponding name in the QS dataframe for each university
    for i,THE_uni in enumerate(THE_name):
        uni, prob=process.extractOne(THE_uni, QS_name, scorer=fuzz.token_sort_ratio)
        if prob>87: #if prob<87, I observed that the algorithm matches diffeent universities!! 97 is a good limit
            THE.set_value(i,'THE_corresponding QS name', uni)
    
    #df1[df1['country']==loc]

I am gonna set QS ranking as reference and do the matching :)


In [310]:
#### martinos version not anymore hehe
def merging_by_name(the,qs):
    # creating two lists with the names of the universities from the two datasets
    THE_name = list(the.THE_name)
    QS_name = list(qs.QS_name)

    #initializing a new column of the THE_df with the corresponding QS name found by the matching function
    # just to control that everything went smoothly
    the['THE_corresponding QS name']='unknown'
    the['prob']='unknown'

    # MATCHING FUNCTION
    # finding the probable corresponding name in the QS dataframe for each university
    for i,THE_uni in enumerate(THE_name):
        uni, prob=process.extractOne(THE_uni, QS_name, scorer=fuzz.token_sort_ratio)
        if prob>87: #if prob<87, I observed that the algorithm matches diffeent universities!! 97 is a good limit
            the['THE_corresponding QS name'][the['THE_name']== THE_uni] = uni
            the['prob'][the['THE_name']== THE_uni] = prob
            #the.set_value(i, 'THE_corresponding QS name', uni)

    # MERGING        
    Unique_df=pd.merge(the,qs, left_on='THE_corresponding QS name', right_on='QS_name', how = 'left')
    return Unique_df

In [311]:
Aus_num_QS = QS_tmp['country'].value_counts().sort_index()['Australia']
Aus_num_THE = THE_tmp['THE_location'].value_counts().sort_index()['Australia']
Aus_num_QS - Aus_num_THE

1

In [312]:
## Just a test, don't consider this
unique_list = []
loc = QS_tmp['country'].value_counts().sort_index().index[0]

THE = THE_tmp[THE_tmp['THE_location'] == loc]
QS = QS_tmp[QS_tmp['country'] == loc]
unique = merging_by_name(THE,QS)
unique
unique['QS_name'].isnull().sum()

/home/zampieri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/zampieri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/zampieri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pa

1

In [313]:
# create a list of df by country which are already merged
unique_list = []
for loc in QS_tmp['country'].value_counts().sort_index().index:
    THE = THE_tmp[THE_tmp['THE_location'] == loc]
    QS = QS_tmp[QS_tmp['country'] == loc]
    unique = merging_by_name(THE,QS)
    unique_list.append(unique)

/home/zampieri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/zampieri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/zampieri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pa

In [314]:
Unique_df = pd.concat(unique_list)

#SOMETHING'S STRANGE IN THE NEIGHBOURHOOD: WE NEED THE DATABUSTERS!
print(QS_tmp.shape)
print(THE_tmp.shape)
print(Unique_df.shape)

(187, 10)
(199, 10)
(199, 22)


In [315]:
list(QS_tmp['country'].value_counts().sort_index().index)

['Australia',
 'Austria',
 'Belgium',
 'Canada',
 'China',
 'Denmark',
 'Finland',
 'France',
 'Germany',
 'Hong Kong',
 'Ireland',
 'Italy',
 'Japan',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Russia',
 'Singapore',
 'South Africa',
 'South Korea',
 'Spain',
 'Sweden',
 'Switzerland',
 'Taiwan',
 'United Kingdom',
 'United States']

In [316]:
missing_num_list_predicted = []
missing_num_list_observed = []
loc_list_num_QS = []
loc_list_num_THE = []

for i,loc in enumerate(QS_tmp['country'].value_counts().sort_index().index):
    
    loc_num_QS = QS_tmp['country'].value_counts().sort_index()[loc]
    loc_list_num_QS.append(loc_num_QS)
    
    loc_num_THE = THE_tmp['THE_location'].value_counts().sort_index()[loc]
    loc_list_num_THE.append(loc_num_THE)
    
    missing_num_list_predicted.append(abs(loc_num_QS - loc_num_THE))
    
    missing_num_list_observed.append(unique_list[i]['QS_name'].isnull().sum())
# end of the for loop


d = {'country': list(QS_tmp['country'].value_counts().sort_index().index),\
     'predicted': missing_num_list_predicted,\
     'observed' : missing_num_list_observed,\
     'QS' : loc_list_num_QS,\
     'THE' : loc_list_num_THE\
    }
df_missing_comparison = pd.DataFrame(data=d)

obs_minus_pred = df_missing_comparison['observed'].sum()-df_missing_comparison['predicted'].sum()
print(obs_minus_pred)
df_missing_comparison

0


,QS,THE,country,observed,predicted
0,9,8,Australia,1,1
1,2,1,Austria,0,1
2,4,4,Belgium,1,0
3,7,6,Canada,1,1
4,7,7,China,0,0
5,3,3,Denmark,0,0
6,2,2,Finland,0,0
7,5,6,France,4,1
8,11,20,Germany,16,9
9,5,5,Hong Kong,0,0


In [299]:
Unique_df[['THE_name', 'QS_name', 'prob','THE_aliases','THE_location']][Unique_df['THE_location']=='South Korea']

,THE_name,QS_name,prob,THE_aliases,THE_location
0,Pohang University of Science and Technology,Pohang University of Science And Technology (P...,91,Pohang University of Science and Technology po...,South Korea
1,Seoul National University,Seoul National University,100,Seoul National University,South Korea
2,Korea Advanced Institute of Science and Techno...,KAIST - Korea Advanced Institute of Science & ...,96,Korea Advanced Institute of Science and Techno...,South Korea
3,Sungkyunkwan University (SKKU),Sungkyunkwan University (SKKU),100,Sungkyunkwan University (SKKU),South Korea


We could for example ignore the accents